In [1]:
import scipy
import numpy as np
import seaborn as sns
import fiona
import rasterio
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.utils.data as torch_data
import sys
from shapely import geometry
from rasterio.mask import mask
from tqdm import tqdm_notebook
from extractor_helper import extractor
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score


In [2]:
import torch


In [3]:
torch.cuda.current_device()

0

Data Import

In [4]:
sat_image = rasterio.open("project/1030010056130F00_MS_Pan_modified_KNN.tif")
sat_points1 = fiona.open("project/Points_MS_Pan_KNN.shp", "r")
sat_points2 = fiona.open("project/AB_points.shp", "r")

test2_img = rasterio.open("Eval/test2/pp_2_sat_modified.tif")
test2_points = fiona.open("Eval/test2/points_2_modified_Copy.shp", "r")

test3_img = rasterio.open("Eval/test3/pp_3_sat_modified.tif")
test3_points = fiona.open("Eval/test3/targets_Copy.shp", "r")

test4_img = rasterio.open("Eval/test4/pp_4_sat_modified_spline.tif")
test4_points = fiona.open("Eval/test4/modified_points_Copy.shp", "r")

In [5]:
class TreesData():
  def __init__(self, X, y):
    super().__init__()
    self.X = torch.tensor(X, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.float32)
    
  def __len__(self):
    return self.X.shape[0]
  
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]
    

In [6]:
class AutoEncoder(nn.Module):
  def __init__(self,encoder,decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    
  def forward(self,x):
    z = self.encoder(x)
    out = self.decoder(z)
    return out

In [7]:
def train(epochs, net, criterion, optimizer,
          train_loader, val_loader, scheduler=None,
          verbose=True, save_dir=None):
  loss_tr =0
  loss_val =0
  net.to(device)

  for epoch in range(1,epochs+1):
      net.train()
      for X,_ in train_loader:
          X = X.to(device)
          pred = net(X)
          loss = criterion(pred,X)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
    
      net.eval()
      for X,_ in val_loader:
          X = X.to(device)
          val_pred = net(X)
          val_loss = criterion(val_pred,X)
#       loss /= len(train_loader)
#       val_loss /= len(val_loader)
      loss_tr +=loss.item()
      loss_val +=val_loss.item()

  
      if scheduler is not None:
          schduler.step()
    
      freq = max(epochs//20,1)
      if verbose and epoch%freq==0:
        print('Epoch {}/{} || Loss:  Train {:.6f} | Validation {:.6f}'.format(epoch, epochs, loss.item(), val_loss.item())) 
  return (loss_tr/epochs), (loss_val/epochs)

In [8]:
encoder = lambda hid,red_size: nn.Sequential(
  
  nn.Conv2d(8,60,3),
  nn.BatchNorm2d(60),
  nn.ReLU(),

  nn.Conv2d(60,200,3),
  nn.BatchNorm2d(200),
  nn.ReLU(),

  nn.Flatten(),
  nn.Linear(200*red_size*red_size,300),
  nn.BatchNorm1d(300),
  nn.ReLU(),

  nn.Linear(300,hid),
  nn.BatchNorm1d(hid),
)

decoder = lambda hid,orginal_size: nn.Sequential(
  nn.Linear(hid,600),
  nn.BatchNorm1d(600),
  nn.ReLU(),

  nn.Linear(600,1000),
  nn.BatchNorm1d(1000),
  nn.ReLU(),

  nn.Linear(1000,orginal_size*orginal_size*8),
  nn.Sigmoid(),
  Unflatten(),
)


In [9]:
class Unflatten(nn.Module):
  def forward(self, input):
    return input.reshape(input.shape[0],8,orginal_size,orginal_size)

In [10]:
from tqdm import tqdm
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

for cut_size in tqdm((range(15,16,2))):    
    for bot_neck in tqdm((range(40,130,20))):

        sm_images1, sm_points1,_ = extractor(sat_image,sat_points1,size=cut_size, normalize=True)
        sm_images2,sm_point2,_= extractor(sat_image,sat_points2,size=cut_size, normalize=True)
        data = np.array(sm_images1+sm_images2)
        data = np.moveaxis(data,3,1)
        targets = np.zeros(shape = data.shape[0])

        X_Train, X_Test, Y_Train, Y_Test = train_test_split(data, targets,test_size=.20, shuffle=True,random_state=12)
        train_data = TreesData(X_Train,Y_Train)
        test_data = TreesData(X_Test,Y_Test)

        orginal_size = X_Test.shape[-1]
        side = lambda x: int(((x - 3)/1)+1)
        red_size  = side(side(orginal_size))

        del X_Train, X_Test,Y_Train,Y_Test,data,targets,sm_images1,sm_images2,sm_point2,sm_points1

        net = AutoEncoder(encoder(bot_neck,red_size), decoder(bot_neck,orginal_size))
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(net.parameters())
        scheduler = None 
        train_loader = torch_data.DataLoader(train_data,batch_size=300, shuffle=True)
        val_loader = torch_data.DataLoader(test_data,batch_size=300,shuffle=True)
        
        del train_data,test_data
        
        lsstr, lssts = train(300, net, criterion, optimizer, train_loader, val_loader,scheduler)
        torch.save(net,f'models/net_cutsize{cut_size}butneck{bot_neck}.pth')




  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 15/300 || Loss:  Train 0.001982 | Validation 0.002055
Epoch 30/300 || Loss:  Train 0.001718 | Validation 0.002102
Epoch 45/300 || Loss:  Train 0.001535 | Validation 0.002168
Epoch 60/300 || Loss:  Train 0.001347 | Validation 0.002303
Epoch 75/300 || Loss:  Train 0.001183 | Validation 0.002494
Epoch 90/300 || Loss:  Train 0.001054 | Validation 0.002639
Epoch 105/300 || Loss:  Train 0.000995 | Validation 0.002596
Epoch 120/300 || Loss:  Train 0.000946 | Validation 0.002761
Epoch 135/300 || Loss:  Train 0.000864 | Validation 0.002789
Epoch 150/300 || Loss:  Train 0.000826 | Validation 0.002948
Epoch 165/300 || Loss:  Train 0.000812 | Validation 0.002901
Epoch 180/300 || Loss:  Train 0.000754 | Validation 0.002883
Epoch 195/300 || Loss:  Train 0.000807 | Validation 0.003036
Epoch 210/300 || Loss:  Train 0.000730 | Validation 0.003016
Epoch 225/300 || Loss:  Train 0.000730 | Validation 0.002951
Epoch 240/300 || Loss:  Train 0.000685 | Validation 0.003067
Epoch 255/300 || Loss:  Train 

/trinity/home/a.shtanchaev/.conda/envs/newone/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type AutoEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/trinity/home/a.shtanchaev/.conda/envs/newone/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Unflatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "

 20%|██        | 1/5 [21:39<1:26:39, 1299.92s/it]

Epoch 15/300 || Loss:  Train 0.001701 | Validation 0.001915
Epoch 30/300 || Loss:  Train 0.001344 | Validation 0.001652
Epoch 45/300 || Loss:  Train 0.001225 | Validation 0.001735
Epoch 60/300 || Loss:  Train 0.001074 | Validation 0.001760
Epoch 75/300 || Loss:  Train 0.000971 | Validation 0.001852
Epoch 90/300 || Loss:  Train 0.000889 | Validation 0.001923
Epoch 105/300 || Loss:  Train 0.000796 | Validation 0.002033
Epoch 120/300 || Loss:  Train 0.000735 | Validation 0.002060
Epoch 135/300 || Loss:  Train 0.000707 | Validation 0.002135
Epoch 150/300 || Loss:  Train 0.000669 | Validation 0.002117
Epoch 165/300 || Loss:  Train 0.000646 | Validation 0.002194
Epoch 180/300 || Loss:  Train 0.000646 | Validation 0.002183
Epoch 195/300 || Loss:  Train 0.000616 | Validation 0.002306
Epoch 210/300 || Loss:  Train 0.000595 | Validation 0.002308
Epoch 225/300 || Loss:  Train 0.000592 | Validation 0.002342
Epoch 240/300 || Loss:  Train 0.000581 | Validation 0.002309
Epoch 255/300 || Loss:  Train 


 40%|████      | 2/5 [43:14<1:04:54, 1298.22s/it]

...


 60%|██████    | 3/5 [1:04:48<43:14, 1297.08s/it]

Epoch 300/300 || Loss:  Train 0.000467 | Validation 0.001969
Epoch 15/300 || Loss:  Train 0.001621 | Validation 0.001742
Epoch 30/300 || Loss:  Train 0.001167 | Validation 0.001526
Epoch 45/300 || Loss:  Train 0.000999 | Validation 0.001344
Epoch 60/300 || Loss:  Train 0.000812 | Validation 0.001354
Epoch 75/300 || Loss:  Train 0.000760 | Validation 0.001373
Epoch 90/300 || Loss:  Train 0.000696 | Validation 0.001407
Epoch 105/300 || Loss:  Train 0.000663 | Validation 0.001462
Epoch 120/300 || Loss:  Train 0.000586 | Validation 0.001427
Epoch 135/300 || Loss:  Train 0.000566 | Validation 0.001547
Epoch 150/300 || Loss:  Train 0.000549 | Validation 0.001554
Epoch 165/300 || Loss:  Train 0.000538 | Validation 0.001623
Epoch 180/300 || Loss:  Train 0.000496 | Validation 0.001576
Epoch 195/300 || Loss:  Train 0.000474 | Validation 0.001594
Epoch 210/300 || Loss:  Train 0.000487 | Validation 0.001573
Epoch 225/300 || Loss:  Train 0.000476 | Validation 0.001635
Epoch 240/300 || Loss:  Train 


 80%|████████  | 4/5 [1:26:23<21:36, 1296.39s/it]

Epoch 300/300 || Loss:  Train 0.000409 | Validation 0.001681
Epoch 15/300 || Loss:  Train 0.001584 | Validation 0.001766
Epoch 30/300 || Loss:  Train 0.001080 | Validation 0.001440
Epoch 45/300 || Loss:  Train 0.000894 | Validation 0.001349
Epoch 60/300 || Loss:  Train 0.000798 | Validation 0.001284
Epoch 75/300 || Loss:  Train 0.000748 | Validation 0.001310
Epoch 90/300 || Loss:  Train 0.000639 | Validation 0.001289
Epoch 105/300 || Loss:  Train 0.000572 | Validation 0.001328
Epoch 120/300 || Loss:  Train 0.000563 | Validation 0.001333
Epoch 135/300 || Loss:  Train 0.000535 | Validation 0.001354
Epoch 150/300 || Loss:  Train 0.000509 | Validation 0.001434
Epoch 165/300 || Loss:  Train 0.000487 | Validation 0.001402
Epoch 180/300 || Loss:  Train 0.000465 | Validation 0.001429
Epoch 195/300 || Loss:  Train 0.000445 | Validation 0.001404
Epoch 210/300 || Loss:  Train 0.000420 | Validation 0.001448
Epoch 225/300 || Loss:  Train 0.000495 | Validation 0.001518
Epoch 240/300 || Loss:  Train 


100%|██████████| 1/1 [1:47:58<00:00, 6478.28s/it]


In [17]:
from ignite.engine import Engine, Events
from ignite.metrics import MeanSquaredError, Loss, RunningAverage

In [18]:
sm_images1, sm_points1,_ = extractor(sat_image,sat_points1,size=7, normalize=True)
sm_images2,sm_point2,_= extractor(sat_image,sat_points2,size=7, normalize=True)
data = np.array(sm_images1+sm_images2)
data = np.moveaxis(data,3,1)
targets = np.zeros(shape = data.shape[0])

X_Train, X_Test, Y_Train, Y_Test = train_test_split(data, targets,test_size=.20, shuffle=True,random_state=12)
train_data = TreesData(X_Train,Y_Train)
test_data = TreesData(X_Test,Y_Test)
train_loader = torch_data.DataLoader(train_data,batch_size=300, shuffle=True)
val_loader = torch_data.DataLoader(test_data,batch_size=300,shuffle=True)

In [19]:
def process_function(engine, batch):
    model.to(device)
    model.train()
    optimizer.zero_grad()
    x, _ = batch
    x = x.to(device)
    x_pred = model(x)

    loss = criterion(x_pred,x)
    loss.backward()
    optimizer.step()
    return loss.item()




In [20]:
def evaluate_function(engine, batch):
    model.eval()
    with torch.no_grad():
        x, _ = batch
        x = x.to(device)
        x_pred = model(x)
        return x_pred, x

In [21]:
trainer = Engine(process_function)
evaluator = Engine(evaluate_function)


orginal_size = X_Test.shape[-1]
side = lambda x: int(((x - 3)/1)+1)
red_size  = side(side(orginal_size))

model = AutoEncoder(encoder(60,red_size), decoder(60,orginal_size))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [36]:
trainer.run(train_loader, max_epochs=200)

Epoch[144] Iteration[28/28] Loss: 0.0001
Epoch[144] Iteration[28/28] Loss: 0.0001
Epoch[144] Iteration[28/28] Loss: 0.0001
Epoch[144] Iteration[28/28] Loss: 0.0001
Epoch[144] Iteration[28/28] Loss: 0.0001
Epoch[144] Iteration[28/28] Loss: 0.000110
Epoch[145] Iteration[28/28] Loss: 0.0001
Epoch[145] Iteration[28/28] Loss: 0.0001
Epoch[145] Iteration[28/28] Loss: 0.0001
Epoch[145] Iteration[28/28] Loss: 0.0001
Epoch[145] Iteration[28/28] Loss: 0.0001
Epoch[145] Iteration[28/28] Loss: 0.000112
Epoch[146] Iteration[28/28] Loss: 0.0001
Epoch[146] Iteration[28/28] Loss: 0.0001
Epoch[146] Iteration[28/28] Loss: 0.0001
Epoch[146] Iteration[28/28] Loss: 0.0001
Epoch[146] Iteration[28/28] Loss: 0.0001
Epoch[146] Iteration[28/28] Loss: 0.000121
Epoch[147] Iteration[28/28] Loss: 0.0001
Epoch[147] Iteration[28/28] Loss: 0.0001
Epoch[147] Iteration[28/28] Loss: 0.0001
Epoch[147] Iteration[28/28] Loss: 0.0001
Epoch[147] Iteration[28/28] Loss: 0.0001
Epoch[147] Iteration[28/28] Loss: 0.000108
Epoch[14

Current run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: 

In [35]:

log_interval = 28
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iteration = (engine.state.iteration - 1) % len(train_loader) + 1
    if iteration % log_interval == 0:
        print("Epoch[{}] Iteration[{}/{}] Loss: {:.6f}".format(engine.state.epoch, iteration, len(train_loader), engine.state.output))

In [ ]:
from ignite.metrics import Loss, CategoricalAccuracy, Precision, Recall


metrics = {
    'avg_loss': Loss(criterion)
}

In [ ]:
from ignite.engine import create_supervised_evaluator

train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)

In [39]:
model.compile()

AttributeError: 'AutoEncoder' object has no attribute 'compile'